# Production Performance - Near Real-Time Analysis 
<h2><span style="color: #117d30;"> Using Azure Cosmos DB  - Azure Synapse Link </span></h2>

# Overview
Consider a typical industry scenario in which a conveyor belt scans or checks the quality of manufactured products and categorizes them into three categories viz. $Good$ OR $Snag$ OR $Reject$. The product quality  whole process gets saved to a local database and gets pushed to the Synapse SQL server instance every night.

Normally, we create an ETL pipeline that runs every night on the above data and creates meaningful insights out of it. So in such a case, to see the overall performance of machines, we need to wait for 24 hours.

But, what if we can get this analysis in near real-time, say every 30 minutes or so, then this will make a huge difference, and can also derive many possibilities for changes.

Here, in this notebook we will showcase the capability of Azure Cosmos DB with Azure Synapse Link and how we can achieve near real time analytics on data. See the workflow below.

# Workflow

![Image alt text](https://dreamdemostorageforgen2.blob.core.windows.net/mfgdemocontainer/notebook-images/htap2.png)

### Scenario
Let's consider where we are getting telemetry data every 15 minutes from 5 machines into our Azure Cosmos DB with the following information :
- Overall Good / Snag / Reject
- Overall Average of Good / (Good + Snag + Reject)
- Timestamp

**Dataset**: Above Quality data coming every 15 minutes from 5 machines, describing the number of items that were Good / Snag / Reject and what is the average production quality.

Total records a day per machine = 1440/15 = 96

5 Machines, 96 * 5 = 480

Records per day = 480

***Database contains data from 24th June 2019 to 30th June 2019.***

Total records in database = 480 * 7 = **3360 records**


### Tools/Techniques: 
- Azure Cosmos DB Analytical store feature and Azure Synapse Link
- Visualization Using Matplotlib


### Notebook Organization 

This notebook uses Azure Synapse Link functionality and retrieves near real-time data from Azure Cosmos DB Analytical Store and performs near real-time analytics on data. Following are key things achieved in the notebook : 

+ How to read data from Azure Cosmos DB Analytical Store
    - Container Name: mfg-mes-quality
    - Azure Cosmos DB Linked Service Name : MESQuality 

+ Aggregate data for each machine over a day.

+ Find out which machine's performance is degrading in near real-time.

## Load data from Azure Cosmos DB Analytical Store into a Spark DataFrame


In [3]:
# Let's load near-real time data from Telemetry storage - Azure Cosmos DB. To do the same, we use modern 
# syntax of connecting to Azure Cosmos DB Analytical Store.
# In this code block, we are connecting to Azure Cosmos DB via Linked Service Connection. You can find
# the same in Synapse Studio -> Data -> Linked (Tab) -> Cosmos DB -> Your linked service name.
# We also specify Cosmos DB Container, which is mfg-mes-quality in this case.


df_ProductionPerformance = spark.read\
    .format("cosmos.olap")\
    .option("spark.synapse.linkedService", "#COSMOS_LINKED_SERVICE#")\
    .option("spark.cosmos.container", "mfg-mes-quality1")\
    .load()

In [4]:
# To proceed further for performing analysis in the Notebook, we need to import few of Python's data processing 
# libraries. Additionally, we will use matplotlib library to plot the required charts.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt; plt.rcdefaults()
from datetime import datetime

## Data Transformation


In [5]:
# Let's load data into Python DataFrame. Here, we convert Spark DataFrame into Pandas DataFrame, which 
# in combination with Matplotlib provides an extensive set of tools for creating visualizations.
# Once we have fetched Telemetry data we convert raw Timestamp values in String to datetime object.
# This allows us perform aggregation as needed over the Time Series.

# converting Spark DataFrame to Pandas for data exploration and visualization 
df_ProductionQuality = df_ProductionPerformance.toPandas()    

# converting a column datetime object column
df_ProductionQuality['ProductionMonth'] = pd.to_datetime(df_ProductionQuality['ProductionMonth'])

## Data Aggregation

-  We will **aggregate data for each day** grouped by machines.  

In [6]:
# We receive Quality data from MES System at regular interval. To better understand this data, we perform grouping on basis of:
# 1. Date
# 2. Machine Instance

# Post grouping, let's see how data looks by displaying entire Aggregated DataFrame.

# setting datetime index on Pandas DataFrame
df_ProductionQuality.set_index(df_ProductionQuality["ProductionMonth"],inplace=True)

# grouping data by machine and aggregating using daily mean
df_AggProductionQuality = df_ProductionQuality.groupby(["MachineInstance"], sort=False).resample('D').mean().reset_index()

display(df_AggProductionQuality.head())

## Tabulating the daywise averages for machines
- The above dataFrame (df_ProdPerfBatch) has aggregated data per day for each machine.

-  We will showcase this using a Table.

- Lets keep a **threshold of 95%**. whenever a machine's daily average falls below 95% it will be marked as critical.

In [7]:
# It's time to visualize our DataFrame and identify what fits our requirements from a business perspective.
# As a business rule, threshold of 95% is minimum acceptable Quality.
# Lower than 95% threshold indicates a problem which may need further inspection.
# For visualizing data, we use matplotlib to draw a table 
# with yellow background indicating a problem.

# plotting a table to see percentage of daily quality data for each machine

cols = df_AggProductionQuality['ProductionMonth'].unique()
index = df_AggProductionQuality['MachineInstance'].unique()
index.sort()
df_AvgMachineProduction = pd.DataFrame(index=index,columns=cols)
df_TableColors = pd.DataFrame(index=index,columns=cols)

rgb_normal = '#01B8AA'
rgb_alert = '#E2B803'

for i,row in df_AggProductionQuality.iterrows():
    r = row['MachineInstance']
    c = row['ProductionMonth']
    avg_str = str(round(row['Avg'],2))[:5]
    if len(avg_str) < 5:
        avg_str+='0'
    df_AvgMachineProduction.loc[r,c] = avg_str
    if(row['Avg']<95):
        df_TableColors.loc[r,c]=rgb_alert
    else:
        df_TableColors.loc[r,c]=rgb_normal

fig, ax = plt.subplots(figsize = (4,4))
ax.axis('off')
ax.axis('tight')
colNames = [dt.strftime("%d %b") for dt in pd.to_datetime(cols)]
the_table = ax.table(cellText=np.array(df_AvgMachineProduction),cellColours=np.array(df_TableColors), cellLoc ='center', colWidths=[0.15]*len(cols) ,rowLabels=index,colLabels=colNames,loc='center')
the_table.auto_set_font_size(False)
the_table.set_fontsize(12)
the_table.scale(3,2)

fig.tight_layout()
plt.show()

## Horizontal Stacked Bar Chart Visualization
- Quality data contains number of items processed into three categories :
    - Good
    - Snag
    - Reject
- We will plot it as a Stack chart.

In [8]:
# Once we have plotted Quality averages, we need to see distribution between elements of Quality, viz, Good, Snag and Reject.
# To plot elements of Quality, we use Horizontally Stacked Bar Charts, with customized starting point to see all values.


# Plotting a horizontal stacked bar chart to show daily 'Good', 'Snag', and 'Reject' outputs for each machine
# Note: These numbers represent the production calculated by averaging each batch in a day and summed across all machines

df_stackPlot = df_ProductionQuality.resample('D').sum().reset_index()

y = df_stackPlot['ProductionMonth']
df_Good = df_stackPlot['Good']
df_Snag = df_stackPlot['Snag']
df_Reject = df_stackPlot['Reject']

#Horizontal Stack Bar Plot
fig, axs = plt.subplots(figsize=(13,7))
y_index = np.arange(len(y))

color_good = ["#02B480"]
color_snag= ["#E2B803"]
color_reject = ["#FD6C53"]

colNames = [dt.strftime("%d-%b") for dt in pd.to_datetime(y)]

bar1 = axs.barh(y_index, df_Good, alpha=0.7, color = color_good, linewidth = 2, edgecolor = color_good, tick_label=colNames)
bar2 = axs.barh(y_index, df_Snag,left=df_Good, alpha=0.7,  color = color_snag,linewidth= 2,edgecolor=color_snag,tick_label=colNames)
bar3 = axs.barh(y_index, df_Reject, left =(df_Snag+df_Good), alpha=0.7,  color = color_reject,linewidth= 2,edgecolor=color_reject,tick_label=colNames)

# setting x limit
min_val = min(df_Good)
max_val = max(df_Good+df_Snag+df_Reject)
diff = max_val - min_val
x_min = 2000 # min_val-diff/2
x_max = 11000 #max_val+diff/2
axs.set_xlim(x_min, x_max)

# writing text inside bars
for i, patch in enumerate(bar1.get_children()):
    bl = patch.get_xy()
    x_cord = x_min/2 +  0.5*patch.get_width() + bl[0]
    y_cord = 0.5*patch.get_height() + bl[1]
    axs.text(x_cord,y_cord, str(int(round(df_Good[i]))), ha='center',va='center')


axs.set_xlabel('Production')
axs.set_ylabel('Date')
fig.suptitle('Overall Production')
plt.legend((bar1[0], bar2[0], bar3[0]), ('Good', 'Snag', 'Reject'))

plt.show()



## Horizontal Bar Chart Visualization
- To get an insight of which machine is behaving abnormaly or showing poor performance, lets compare all of them using a bar chart.
- We are calculating mean of each machine's production AVG for all seven days.

In [9]:
# Up until now, we have been visualizing daily metrics. To interprete exact problem,
# we also need to see how each machine Quality output looks like. By combining data from both
# sides, we can pinpoint a specific machine that appears to be under performing.


# plotting bar graph chart to show total quality of production for each machine


#Bar Chart
df_MachineTotalAvg = df_ProductionQuality.groupby(["MachineInstance"], sort=True).mean()

# creating a list of avg production of each machine
performance = df_MachineTotalAvg['Avg']
names= df_MachineTotalAvg.index.tolist()

# plotting bar graph
plt.figure(figsize=(13,7))
y_pos = np.arange(len(names))

edge_colour = ["#FFA132"]
color_map = ['#FFA132', '#FFAA46', '#FFB35A', '#FFBC6D', '#FFC581', '#FFCE94', '#FFD7A8']
plt.barh(y_pos, performance, align='center', alpha=1,  color = color_map, edgecolor=color_map)
plt.yticks(y_pos, names)

perf_min = min(performance)
perf_max = max(performance)
perf_diff = (perf_max - perf_min)/2
plt.xlim(perf_min-perf_diff, perf_max+perf_diff)

plt.xlabel('Percentage')
plt.ylabel('Machines')
plt.title('Production Performance of Machines')

plt.show()

In [10]:
# Let's visualize correlation between Quality of production of all machines per day against number of Good pieces produced per day.
# For doing the same, we are going to use Date on X-Axis. Stackbars will represent Average quality output of all machines.
# Line overlapped over Stackbars will represent number of Good pieces produced across all machines for that day.

# plotting graph to show the correlation between daily overall quality and average quality of all batches in a day 


df_barPlot = df_AggProductionQuality.set_index(df_AggProductionQuality["ProductionMonth"]).resample('D').mean().reset_index()
df_linePlot = df_AggProductionQuality.set_index(df_AggProductionQuality["ProductionMonth"]).resample('D').sum().reset_index()

x = df_barPlot['ProductionMonth'].tolist()
x_Names = [dt.strftime("%d-%b") for dt in pd.to_datetime(x)]
x_pos = np.arange(len(x))

y = df_barPlot['Avg'].tolist()
y_min = min(y)
y_max = max(y)
y_diff = (y_max - y_min)

y2 = df_linePlot['Good'].tolist()


# plotting graph
fig, ax1 = plt.subplots(figsize=(13,7))
color_map = ['#FFA132', '#FFAA46', '#FFB35A', '#FFBC6D', '#FFC581', '#FFCE94', '#FFD7A8']

# bar plot
p1 = ax1.bar(x_pos, y, align='center', alpha=1, color = color_map,tick_label=x_Names)
ax1.set_xticks(x_pos,x_Names)
ax1.set_ylim(y_min-y_diff, y_max+y_diff/2)

# line plot
line_color = '#02C480'
ax2 = ax1.twinx()
p2 = ax2.plot(x_pos,y2,color = line_color)
ax2.set_ylabel('Quality')

ax1.set_ylabel('Average Percentage of Availability')
ax1.set_xlabel('Date')
fig.suptitle("Overall Quality")
plt.legend((p1[0], p2[0]), ('Quality', 'Good'))
plt.show()

## Visualizing Average Availability for all Machines till 30th June 7:00 AM



In [11]:
# Let's see what is happening currently. Through Azure Synapse Link feature, we can
# perform near-real time analytics on data. 
# Let's plot a graph showing average Quality output of all machines for the last few hours.


df_AvailabilityChartData = df_ProductionPerformance.toPandas()
df_AvailabilityChartData.sort_values('ProductionMonth', inplace=True)
df_RecentRecords = df_AvailabilityChartData.tail(60)
df_RecentRecords = df_RecentRecords.groupby(["ProductionMonth"], sort=True).mean()

performance = df_RecentRecords['Avg'].tolist()
names = df_RecentRecords.index.tolist()
perf_min = min(performance)
perf_max = max(performance)
perf_diff = (perf_max - perf_min)/2
# performance = [x for x in performance]

# plotting bar graph
plt.figure(figsize=(13,7))
y_pos = np.arange(len(names))
colNames = [dt.strftime("%H:%M") for dt in pd.to_datetime(names)]
color_map=['#02B480','#02B480','#02B480','#E2B803','#02B480','#02B480','#02B480','#F4B71C','#F6A629','#F79537','#F98444','#FB7351']
plt.bar(y_pos, performance, align='center', alpha=1, color = color_map, edgecolor=color_map)
plt.xticks(y_pos, colNames)
plt.ylim(perf_min-perf_diff, perf_max+perf_diff)
plt.ylabel('Average Percentage of Quality')
plt.xlabel('Time of the day ({})'.format(pd.to_datetime(names)[0].strftime("%d-%b")))
plt.title("Machines Average Quality (Till 30th June 7:00 AM)")

plt.show()